# Merge home data


In [ ]:
import pandas as pd
import numpy as np

home_weather_locations_file_path = "home_weather_locations.parquet"
home_data_file_path = "home_data.parquet"

In [ ]:
%%time
# read home data from Parquet file
try:
    df_homes_weather = pd.read_parquet(
        home_weather_locations_file_path, 
        engine='pyarrow',
        use_nullable_dtypes=True
        )
    print("File was successfully read without specifying compression codec.")
except Exception as e:
    print(f"Error reading file: {e}")


In [ ]:
df_homes = df_homes_weather

In [ ]:
df_homes

## Merge other home data

In [ ]:
# TO DO: boiler type

## Write merged DataFrame to file

In [ ]:
%%time 
df_homes.to_parquet(home_data_file_path, index=True, engine='pyarrow')